In [96]:
import pandas as pd
import requests


url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EDDB/2021-11-25T00:00/2021-11-25T11:00"

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': "8f9f83bab9msh9cbc143b90ebf53p169e3ejsn9597fff30d82"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

#print(response.text)

In [97]:

from IPython.display import JSON
JSON(response.json())

<IPython.core.display.JSON object>

In [98]:
flight_arrivals = response.json()#['list']
#flight_arrivals

In [99]:
#flight_arrivals['arrivals'][33]['arrival']['scheduledTimeLocal']#[0]#['number']#.keys()

In [100]:
#flight_arrivals['arrivals'][33][ 'number']

In [101]:
flight_arrivals['arrivals'][33]['departure']['airport']['name']#.keys()#['airline']['name']

'Istanbul'

In [114]:

departure_from = []
flight_name = []
flight_number = []
arrival_time = []

for i in flight_arrivals['arrivals']:
    arrival_time.append(i['arrival']['scheduledTimeLocal'])
    flight_number.append(i[ 'number'])
    flight_name.append(i['airline']['name'])
    departure_from.append(i['departure']['airport']['name'])
    
    
    
df = pd.DataFrame({
    'flight_name':flight_name,
    'flight_number':flight_number,
    'departure_from':departure_from,
    'arrival_time':arrival_time   
})


In [115]:
df.head()

,flight_name,flight_number,departure_from,arrival_time
0,Eurowings,EW 12,Cologne,2021-11-25 07:20+01:00
1,Eurowings,EW 2002,Stuttgart,2021-11-25 07:30+01:00
2,Eurowings,EW 9048,Duesseldorf,2021-11-25 07:35+01:00
3,Air Baltic,BT 211,Riga,2021-11-25 07:55+01:00
4,easyJet,U2 5632,Bâle/Mulhouse,2021-11-25 07:55+01:00


## code from Joan

In [123]:
airport_icoa = "EDDB"
to_local_time = "2021-10-04T20:00"
from_local_time = "2021-10-05T08:00"

url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icoa}/{to_local_time}/{from_local_time}"

querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"false","withLocation":"false"}

arrivals_berlin = response.json()['arrivals']

def get_flight_info(flight_json):
    # terminal
    try: terminal = flight_json['arrival']['terminal']
    except: terminal = None
    # aircraft
    try: aircraft = flight_json['aircraft']['model']
    except: aircraft = None

    return {
        'dep_airport':flight_json['departure']['airport']['name'],
        'sched_arr_loc_time':flight_json['arrival']['scheduledTimeLocal'],
        'terminal':terminal,
        'status':flight_json['status'],
        'aircraft':aircraft,
        'airport_icoa':airport_icoa
    }


import pandas as pd
arrivals_berlin = pd.DataFrame([get_flight_info(flight) for flight in arrivals_berlin])
arrivals_berlin.head()

,dep_airport,sched_arr_loc_time,terminal,status,aircraft,airport_icoa
0,Cologne,2021-11-25 07:20+01:00,1,Unknown,Boeing 737-800,EDDB
1,Stuttgart,2021-11-25 07:30+01:00,1,Unknown,Airbus A319,EDDB
2,Duesseldorf,2021-11-25 07:35+01:00,1,Unknown,Airbus A320,EDDB
3,Riga,2021-11-25 07:55+01:00,1,Unknown,Airbus A220-300,EDDB
4,Bâle/Mulhouse,2021-11-25 07:55+01:00,1,Unknown,Airbus A319,EDDB


## inserting this value to Databases

In [117]:
import mysql.connector as connection
import pandas as pandas
import numpy as np

arr = np.array(df)

try:
    #Establish a new connection to the database created above
    mydb = connection.connect(host="localhost", database = 'gan',user="root", passwd="Jaanu123", use_pure=True)
    cursor = mydb.cursor() #create a cursor to execute queries
    # Inserting data
    for val in arr:
        data = tuple(val)
        cursor.execute(f"INSERT INTO flight_table values {data}")
    
    print("Values inserted!!")
    mydb.commit()
    cursor.close()
    mydb.close()

except Exception as e:
    mydb.close()
    print(str(e))

Values inserted!!
